http://www.mashangxue123.com/tensorflow/tf2-tutorials-quickstart-advanced.html

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D
from tensorflow.keras import Model

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
# 添加一个通道维度
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [9]:
# 使用tf.data批处理和随机打乱数据集
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [10]:
# 使用keras类型子类API构建tf.keras模型
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [11]:
# 选择优化器和损失函数进行训练
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [12]:
# 选择指标以衡量模型的损失和准确性
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [13]:
# 使用tf.GradientTape训练模型
@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels,predictions)

In [14]:
# 测试模型
@tf.function
def test_step(images,labels):
    predictions = model(images)
    t_loss = loss_object(labels,predictions)
    
    test_loss(t_loss)
    test_accuracy(labels,predictions)

In [15]:
EPOCHS = 5
for epoch in range(EPOCHS):
    for images,labels in train_ds:
        train_step(images,labels)
    for test_images,test_labels in test_ds:
        test_step(test_images,test_labels)
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
        epoch+1,
        train_loss.result(),
        train_accuracy.result()*100,
        test_loss.result(),
        test_accuracy.result()*100
    ))

W0313 18:55:04.164247 44552 base_layer.py:1814] Layer my_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Epoch 1, Loss: 0.13106530904769897, Accuracy: 96.04166412353516, Test Loss: 0.05518670752644539, Test Accuracy: 98.25
Epoch 2, Loss: 0.08568522334098816, Accuracy: 97.41083526611328, Test Loss: 0.05494958534836769, Test Accuracy: 98.2550048828125
Epoch 3, Loss: 0.06459937244653702, Accuracy: 98.0250015258789, Test Loss: 0.05390775576233864, Test Accuracy: 98.30667114257812
Epoch 4, Loss: 0.051729246973991394, Accuracy: 98.40875244140625, Test Loss: 0.055303994566202164, Test Accuracy: 98.35250091552734
Epoch 5, Loss: 0.04347982630133629, Accuracy: 98.65633392333984, Test Loss: 0.056907981634140015, Test Accuracy: 98.31800079345703
